5. 삼성 전자 주식 다음달 가격 얼마일까? (회귀)
8. 비트코인 다음 달 어떻게 될까? (회귀)
16. 전날 비트 코인 상승하면 다음날 주가와의 상관관계(correlation)

1. 목표 설정
  1. Y : 미래 30일의 국내 특정 주식과 비트코인의 종가
  1. X : 최근 60일간의 데이터
    1. step : 하루 간격
    1. 양 : 1 년(2018년 6월 ~2019년 6월)
1. 데이터 획득
  1. 국내 기업 1개, 비트 코인 
1. 데이터 정제
  1. 누락
1. 데이터 전처리  
  1. 특수기호, 문자 > 수치 변환
  1. 두 데이터 날짜 기준으로 통합(axis=1)
    1. 외국(2018-01-01), 국내(2018.1.1),(18.1.1)
    1. 날짜 포맷을 (YYYY-MM-DD) 으로 변환 후 통합
1. 비트코인 가격 변화와 주가와의 상관관계(correlation)
  1.cov()
1. 입력X /출력Y 분리
  1. m개의 x,y 쌍
  1. x(m, 60, 15) (국내증시컬럼 + 비트코인컬럼) > reshape > (m, 60*15)
  1. y(m, 30, 2) > (m, 30 * 2)  
1. 학습용/검증용(test) 분리
  1. 섞으면 안되므로 직접 분리
  1. 가장 과거의 데이터 10% 를 검증용으로 사용
1. 모델 선정
  1. Ridge 모델. 학습용 데이터로 학습
  1. 성능 측정
1. 그래프로 시각화
  1. 과거
  1. 미래의 예측 predict( x(1, 60*15) )  > return  >  y_hat(1, 30 * 2)
     1. 미래 예측(1, 30 * 2) > (30,2) > 국내(30), 비트코인(30) > 그래프로 그리기

국내 기업 데이터 획득 6/1/2018 ~ 6/1/2019

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
#주소중간에 _day 가 추가됩니다
samsung_url='https://finance.naver.com/item/sise_day.nhn?code=005930'

In [18]:
tables = pd.read_html(samsung_url+'&page='+str(1))
len(tables)
samsung = tables[0]
samsung.shape

(15, 7)

In [31]:
samsungs = []
for i in range(1, 28):
    tables = pd.read_html(samsung_url+'&page='+str(i))
    samsung = tables[0]
    samsungs.append(samsung)
len(samsungs)

27

In [32]:
samsung = pd.concat(samsungs, axis=0)
samsung.shape

(405, 7)

In [36]:
samsung.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
10,2018.05.31,50700.0,1200.0,50400.0,50800.0,49850.0,63491109.0
11,2018.05.30,49500.0,1800.0,51300.0,51500.0,49100.0,20498098.0
12,2018.05.29,51300.0,1000.0,52200.0,52500.0,51300.0,8480437.0
13,2018.05.28,52300.0,400.0,52500.0,53000.0,52000.0,9787820.0
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
bitcoin_url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/'
bitcoin_url +='?start=20180601&end=20190601'
tables = pd.read_html(bitcoin_url)
bitcoin = tables[0]
len(tables)

2

In [45]:
bitcoin

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,"Jun 01, 2019",8573.84,8625.60,8481.58,8564.02,22488303544,151890214310
1,"May 31, 2019",8320.29,8586.66,8172.55,8574.50,25365190957,152059886414
2,"May 30, 2019",8661.76,9008.31,8221.27,8319.47,29246528551,147525141134
3,"May 29, 2019",8718.59,8755.85,8482.73,8659.49,23473479966,153537047655
4,"May 28, 2019",8802.76,8807.02,8634.72,8719.96,24226919267,154590202287
5,"May 27, 2019",8674.07,8907.17,8668.70,8805.78,27949839564,156093754762
6,"May 26, 2019",8055.21,8687.52,7924.67,8673.22,26677970091,153725486067
7,"May 25, 2019",7991.89,8117.93,7965.98,8052.54,22256813107,142708188838
8,"May 24, 2019",7881.70,8140.72,7824.45,7987.37,25919126991,141539115575
9,"May 23, 2019",7677.27,7943.79,7533.20,7881.85,24457107820,139654107035
